In [26]:
import numpy as np

In [27]:
my_label = [[0, 2, 1, 0], 
            [0, 2], 
            [1, 1, 0, 0, 0], 
            [0, 2, 2, 0], 
            [3], 
            [0, 0, 2], 
            [0, 0, 2, 0, 0], 
            [0, 0, 0, 0, 0, 1], 
            [0, 2], 
            [1, 1, 2, 0, 1, 0],
            [0, 1, 0, 0, 2, 0],
            [1, 0, 1, 0, 1, 0, 0, 2],
            [0, 1],
            [0, 2, 0, 2, 0],
            [1, 0],
            [0, 0, 0, 2, 0],
            [0],
            [0, 0, 0, 0, 0, 0, 0],
            [0, 0],
            [0, 0],
            [0, 0, 0, 0]]

In [28]:
Yue_label = [[0, 2, 1, 0], 
            [0, 2], 
            [1, 1, 0, 2, 2], 
            [0, 2, 2, 0], 
            [3], 
            [0, 0, 2], 
            [0, 0, 2, 0, 0], 
            [0, 0, 0, 0, 0, 1], 
            [0, 2], 
            [1, 1, 2, 0, 1, 0],
            [0, 1, 0, 0, 2, 0],
            [1, 0, 1, 1, 1, 0, 0, 2],
            [0, 1],
            [0, 1, 0, 2, 0],
            [1, 0],
            [0, 0, 0, 2, 0],
            [0],
            [0, 0, 0, 0, 0, 0, 0],
            [0, 0],
            [0, 0],
            [0, 0, 0, 0]]

In [29]:
Paul_label = [[0, 2, 1, 0], 
             [0, 2], 
             [0, 1, 0, 2, 0], 
             [3, 2, 2, 3], 
             [3], 
             [0, 0, 2], 
             [0, 1, 2, 2, 0], 
             [0, 0, 0, 0, 0, 1], 
             [0, 2], 
             [1, 1, 2, 0, 2, 0],
             [0, 1, 0, 0, 2, 0],
             [0, 0, 1, 1, 1, 0, 0, 2],
             [0, 1],
             [0, 1, 0, 2, 0],
             [1, 0],
             [0, 0, 0, 2, 0],
             [0],
             [0, 0, 0, 0, 0, 0, 0],
             [0, 0],
             [1, 0],
             [0, 0, 0, 0]]

In [30]:
print(len(Yue_label))

21


In [31]:
from sklearn.metrics import cohen_kappa_score

In [32]:
my_label_flat = []
yue_label_flat = []

for l in my_label:
    my_label_flat.extend(l)
    
for y in Paul_label:
    yue_label_flat.extend(y)

cohen_kappa_score(my_label_flat, yue_label_flat)

0.7490261547022816

In [33]:
def fleiss_kappa(table, method='fleiss'):
    """Fleiss' and Randolph's kappa multi-rater agreement measure

    Parameters
    ----------
    table : array_like, 2-D
        assumes subjects in rows, and categories in columns
    method : str
        Method 'fleiss' returns Fleiss' kappa which uses the sample margin
        to define the chance outcome.
        Method 'randolph' or 'uniform' (only first 4 letters are needed)
        returns Randolph's (2005) multirater kappa which assumes a uniform
        distribution of the categories to define the chance outcome.

    Returns
    -------
    kappa : float
        Fleiss's or Randolph's kappa statistic for inter rater agreement
    """
    table = 1.0 * np.asarray(table)   #avoid integer division
    n_sub, n_cat =  table.shape
    n_total = table.sum()
    n_rater = table.sum(1)
    n_rat = n_rater.max()
    #assume fully ranked
    assert n_total == n_sub * n_rat

    #marginal frequency  of categories
    p_cat = table.sum(0) / n_total

    table2 = table * table
    p_rat = (table2.sum(1) - n_rat) / (n_rat * (n_rat - 1.))
    p_mean = p_rat.mean()

    if method == 'fleiss':
        p_mean_exp = (p_cat*p_cat).sum()
    elif method.startswith('rand') or method.startswith('unif'):
        p_mean_exp = 1 / n_cat

    kappa = (p_mean - p_mean_exp) / (1- p_mean_exp)
    return kappa

In [34]:
table = []
major_agreement = 0
for m, y, p in zip(my_label, Yue_label, Paul_label):
    assert len(m) == len(y) == len(p)
    for x, y, z in zip(m, y, p):
        set_length = len(set([x, y, z]))
        if set_length == 3:
            major_agreement += 0
        elif set_length == 2:
            major_agreement += 2
        elif set_length == 1:
            major_agreement += 3
        
        label = [0, 0, 0, 0]
        label[x] += 1
        label[y] += 1
        label[z] += 1
        table.append(label)

In [35]:
fleiss_kappa(table)

0.8172928142600732

In [36]:
major_agreement / (len(table) * 3)

0.9512195121951219

In [37]:
len(table)

82